In [ ]:
import csv
import pandas as pd
import sys
import time
import numpy as np
from collections import defaultdict

In [ ]:
'''
The file is like 

'ua', 'tel_isAttack', 'renderer', 'vendor', 'flowLabel', 'bfp', 'num_distinct'

'''

In [ ]:
# We remove the company name
company = ['a']
for company_num in range(len(company)):
    filename = './'+ company[company_num] + '_new_table_5' + '_ua_bfp' + '.csv'
    file = open(filename,'r',encoding='utf-8',newline='')
    reader_lines = csv.reader(file)
    i = 0
    test = set()
    number = defaultdict(int)
    for raw_data in reader_lines:
        test.add(raw_data[1])
        number[len(raw_data)] += 1
        if len(raw_data) != 7:
            print(raw_data)
        if i < 100:
            print(raw_data)
            i += 1
    print(test)
    print(number)

In [ ]:
company = ['a','b','c','d','e','f','g','h','i','j','k','l','m','n']

simple_keep_arr = []
simple_mimic_arr = []
vm_arr = []
flag_sum_dic = defaultdict(int)

bfp_simple_keep_arr = []
bfp_simple_mimic_arr = []
bfp_vm_arr = []
bfp_flag_sum_dic = defaultdict(int)


for company_num in range(len(company)):
    
    # change name
#     filename = './'+ 'table_5_'+ company[company_num] + '_sample' + '.csv'
    filename = './'+ company[company_num] + '_new_table_5' + '_ua_bfp' + '.csv'
    file = open(filename,'r',encoding='utf-8',newline='')
    reader_lines = csv.reader(file)
    sum_attack = 0
    sum_normal = 0
    bfp = [{} for i in range(6)]
    # This is a demo list about how we create, we can not show the full list
    botua = ["Requests", "bot", "Bot", "PhantomJS", "Python", "Java", 'python']
    botvm = ['VMware','Unknown', 'Virtual']
    test = 0
    for raw_data in reader_lines:
#         print(raw_data)
        if raw_data[0] == 'ua' or len(raw_data) != 7:
            continue
        if raw_data[1] == 'False':
            sum_normal += float(raw_data[6])
        else:
            sum_attack += float(raw_data[6])
        
        
        bfp_arr = raw_data[5].split('-')[1:]
        for i, ele in enumerate (bfp_arr):
            if len(ele) != 8:
                bfp_arr[i] = 'Missing'
        if len(bfp_arr) == 0:
            bfp_arr = ['Missing' for i in range(6)]

        for i in range(len(bfp)):
            if bfp_arr[i] not in bfp[i]:
                if raw_data[1] == 'False':
                    bfp[i][bfp_arr[i]] = [0.0, float(raw_data[6])]
                else:
                    bfp[i][bfp_arr[i]] = [float(raw_data[6]), 0.0]
            else:
                if raw_data[1] == 'False':
                    bfp[i][bfp_arr[i]][1] += float(raw_data[6])
                else:
                    bfp[i][bfp_arr[i]][0] += float(raw_data[6])       
    file = open(filename,'r',encoding='utf-8',newline='')
    reader_lines = csv.reader(file)
    simple_keep = 0
    simple_mimic = 0
    vm = 0
    flag_dic = defaultdict(int)
    bfp_simple_keep = 0
    bfp_simple_mimic = 0
    bfp_vm = 0
    bfp_flag_dic = defaultdict(int)
    
    flag_val = [0 for i in range(6)]
    
    # We mixed some number to the normal number
    # This value is different from different company we analyze 
    # by other resource, we can not show here
    for i in range(6):
        tem = []
        for ele in bfp[i]:
            if bfp[i][ele][0] > 0.5:
                tem.append(bfp[i][ele][0])
        flag_val[i] = np.mean(tem)
    
    for raw_data in reader_lines:
        if raw_data[0] == 'ua' or len(raw_data) != 7:
            continue
        if raw_data[1] != 'False':
            bfp_arr = raw_data[5].split('-')[1:]
            for i, ele in enumerate (bfp_arr):
                if len(ele) != 8:
                    bfp_arr[i] = 'Missing'
            if len(bfp_arr) == 0:
                bfp_arr = ['Missing' for i in range(6)]
            if all(feature == 'Missing' for feature in bfp_arr):
                if any(name in raw_data[0] for name in botua)  :
                    simple_keep += float(raw_data[6])
                    bfp_simple_keep += 1
                else:
                    simple_mimic += float(raw_data[6])
                    bfp_simple_mimic += 1
            elif any(name in raw_data[3] for name in botvm) or any(name in raw_data[2] for name in botvm):
                vm += float(raw_data[6])
                bfp_vm += 1
            else:
                #block, keep, mimic, random, grey 
                # We need combine the feature together
                flag = [0, 0, 0, 0, 0]
                # This part we replace to a simple logic
                # to make sure this code can run 
                # otherwise we need the function from other file
                # We can not show that file. we has other
                # feature to decide whether it is mimic/ random
                # We can give a example about that
                # For mimic --> inconsistent betweeen feature
                # For random --> We find special pattern how attacker
                # give the random value, like small version, verndor 
                # random memory
                for i in range(6):
                    if bfp_arr[i] == 'Missing':
                        flag[0] = 1
                    elif bfp[i][bfp_arr[i]][1] < 5:
                        flag[3] = 1
                    elif bfp[i][bfp_arr[i]][0] < flag_val[i] * 2:
                        flag[2] = 1
                    elif bfp[i][bfp_arr[i]][0] > flag_val[i] * 3:
                        flag[1] = 1
                    else:
                        flag[4] = 1
                flag = tuple(flag)
                flag_dic[flag] += float(raw_data[6])
                bfp_flag_dic[flag] += 1
    print(company[company_num], '==========================================================================')
    print('attack:', sum_attack, 'normal:',sum_normal)
    print('traffic', 'bfp')
    print("simple_keep",simple_keep,bfp_simple_keep,"simple_mimic",simple_mimic,bfp_simple_mimic,"vm",vm,bfp_vm)
    grey = 0
    bfp_grey = 0
    for ele in flag_dic:
        if ele[4] != 0:
            grey += flag_dic[ele]
            bfp_grey += bfp_flag_dic[ele]
    print("grey", grey, bfp_grey)
    flag_name = ["Block", "Keep", "Mimic", "Random"]
    for ele in flag_dic:
        flag_sum_dic[ele] += flag_dic[ele]
        bfp_flag_sum_dic[ele] += bfp_flag_dic[ele]
        
        strategy = ''
        if ele[4] == 0:
            for i in range(len(ele)):
                if ele[i] == 1:
                    if strategy != '':
                        strategy += '+'
                    strategy += flag_name[i]
            print(strategy,flag_dic[ele], bfp_flag_dic[ele])
            
    simple_keep_arr.append(simple_keep)
    simple_mimic_arr.append(simple_mimic)
    vm_arr.append(vm)
    
    bfp_simple_keep_arr.append(bfp_simple_keep)
    bfp_simple_mimic_arr.append(bfp_simple_mimic)
    bfp_vm_arr.append(bfp_vm)
    

print('SUM', '*****************************************************************************')
traffic_sum = 0
bfp_sum = 0
traffic_sum += sum(simple_keep_arr) + sum(simple_mimic_arr) + sum(vm_arr)
bfp_sum += sum(bfp_simple_keep_arr) + sum(bfp_simple_mimic_arr) + sum(bfp_vm_arr)
for ele in flag_sum_dic:
    traffic_sum += flag_sum_dic[ele]
for ele in bfp_flag_sum_dic:
    bfp_sum += bfp_flag_sum_dic[ele]

print('traffic sum', traffic_sum)
print('bfp sum', bfp_sum)
print('traffic  ','%  ', 'bfp', '%' , '# of traffic in per bfp')
print("simple_keep",sum(simple_keep_arr),sum(simple_keep_arr)/traffic_sum,sum(bfp_simple_keep_arr),sum(bfp_simple_keep_arr)/bfp_sum, sum(simple_keep_arr)/sum(bfp_simple_keep_arr)) 
print("simple_mimic",sum(simple_mimic_arr),sum(simple_mimic_arr)/traffic_sum,sum(bfp_simple_mimic_arr),sum(bfp_simple_mimic_arr)/bfp_sum, sum(simple_mimic_arr)/sum(bfp_simple_mimic_arr)    )
print("vm",sum(vm_arr),sum(vm_arr)/traffic_sum,sum(bfp_vm_arr),sum(bfp_vm_arr)/bfp_sum, sum(vm_arr)/sum(bfp_vm_arr))
grey = 0
bfp_grey = 0
for ele in flag_sum_dic:
    if ele[4] != 0:
        grey += flag_sum_dic[ele]
        bfp_grey += bfp_flag_sum_dic[ele]
print("grey", 'traffic', grey,grey/traffic_sum)
print(bfp_grey, bfp_grey/bfp_sum, grey/ bfp_grey)
flag_name = ["block", "keep", "mimic", "random"]
output_dic = {}
# print(flag_sum_dic)
for ele in flag_sum_dic:
    strategy = ''
    if ele[4] == 0:
        for i in range(len(ele)):
            if ele[i] == 1:
                if i == 1 and sum(ele) > 1:
#                     print('in')
                    continue
                if strategy != '':
                    strategy += '+'
                strategy += flag_name[i]
        if strategy not in output_dic:
            output_dic[strategy] = [flag_sum_dic[ele], bfp_flag_sum_dic[ele]]
        else:
            output_dic[strategy][0] += flag_sum_dic[ele]
            output_dic[strategy][1] +=  bfp_flag_sum_dic[ele]
for ele in output_dic:
    print('-------------------')
    print(ele, 'traffic', output_dic[ele][0], output_dic[ele][0]/traffic_sum)
    print(output_dic[ele][1], output_dic[ele][1]/bfp_sum, output_dic[ele][0]/output_dic[ele][1] )
          

    

    

In [ ]:
company = ['a','b','c','d','e','f','g','h','i','j','k','l','m','n']
# Account take over 1 / Fraud 2/ Fake account 3/ Scraping 4/ Giftcard 5
attack_type = defaultdict(str)

attack_type['Account.Create'] = 'Fake'
attack_type['Account.Recover'] = 'Account'
attack_type['Authenticate.Login'] = 'Account'
attack_type['Authenticate.Login.Alexa'] = 'Account'
attack_type['CreditCard.ViewHistory'] = "Scraping"
attack_type['Shop.ApplyGiftCard'] = "Giftcard"
attack_type['Shop.Order'] = "Fraud"
attack_type['Account.CheckEligibility'] = 'Scraping'
attack_type['FinancialServices.Account.Apply'] = 'Fake'
attack_type['FinancialServices.OFX'] = 'Fraud'
attack_type['FinancialServices.Loan.Personal.Apply'] = 'Fraud'
attack_type['FinancialServices.Money.Send'] = 'Fraud'
attack_type['FinancialServices.Money.Transfer'] = 'Fraud'
attack_type['Quote.Request'] = 'Scraping'
attack_type['ofx'] = 'Fraud'
attack_type['Profile.Create'] = 'Fake'
attack_type['Account.ResetPassword'] = 'Account'
attack_type['Authenticate.Logout'] = 'Account'
attack_type['Flight.Status'] = 'Scraping'
attack_type['GiftCard.PurchaseCard'] = 'Giftcard'
attack_type['Loyalty.ViewAccount'] = 'Scraping'
attack_type['Search.Fare'] = 'Scraping'
attack_type['Search.Flight'] = 'Scraping'
attack_type['Account.ForgotPassword'] = 'Account'
attack_type['Login'] = 'Account'
attack_type['Search'] = 'Scraping'
attack_type['TravelInfo'] = 'Scraping'
attack_type['Account.ChangePassword'] = 'Account'
attack_type['Authenticate.Login.2FA'] = 'Account'
attack_type['Search.Product'] = 'Scraping'
attack_type['Shop.AddToCart'] = 'Fraud'
attack_type['CreditCard.ApplyToAccount'] = 'Fraud'
attack_type['GiftCard.CheckBalance'] = "Giftcard"
attack_type['GiftCard.MakePurchaseWithCard'] = "Giftcard"
attack_type['MailingList.CreatePassword'] = "Fake"
attack_type['MailingList.Signup'] = "Fake"
attack_type['Profile.Update'] = "Fake"
attack_type['Shop.Checkout'] = "Fraud"
attack_type['Shop.Return'] = "Fraud"
attack_type['CreditCard.Apply'] = "Fraud"
attack_type['Shop.MakePayment'] = "Fraud"
attack_type['Profile.View'] = "Scraping"

tools = {
    'Giftcard' : [0,0,0],
    'Fake': [0,0,0],
    'Fraud': [0,0,0],
    "Scraping":[0,0,0],
    "Account":[0,0,0],
}

kl_dic = {}
unique_dic = {}
for company_num in range(len(company)):
    
    filename = './'+ company[company_num] + '_new_table_5' + '_ua_bfp' + '.csv'
    file = open(filename,'r',encoding='utf-8',newline='')
    reader_lines = csv.reader(file)
    # This is a demo list about how we create, we can not show the full list
    botua = ["Requests", "bot", "Bot", "PhantomJS", "Python", "Java", 'python']
    botvm = ['VMware','Unknown', 'Virtual']
    simple_tool = defaultdict(int)
    browser_tool = defaultdict(int)
    vm_tool = defaultdict(int)
    
    
    bfp_dic = {
        'Giftcard' : {},
        'Fake': {},
        'Fraud': {},
        "Scraping":{},
        "Account":{},
    }
    
    for raw_data in reader_lines:
        if raw_data[0] == 'ua' or len(raw_data) != 7:
            continue
        att_type = attack_type[raw_data[4]]
        bfp = raw_data[0] + raw_data[5]
        if att_type in bfp_dic:
            if bfp not in bfp_dic[att_type]:
                if raw_data[1] == 'False':
                    bfp_dic[att_type][bfp] = [0.0, float(raw_data[6])]
                else:
                    bfp_dic[att_type][bfp] = [float(raw_data[6]), 0.0]
            else:
                if raw_data[1] == 'False':
                    bfp_dic[att_type][bfp][1] += float(raw_data[6])
                else:
                    bfp_dic[att_type][bfp][0] += float(raw_data[6])
        
        
        if raw_data[1] != 'False':
            bfp_arr = raw_data[5].split('-')[1:]
            for i, ele in enumerate (bfp_arr):
                if len(ele) != 8:
                    bfp_arr[i] = 'Missing'
            if len(bfp_arr) == 0:
                bfp_arr = ['Missing' for i in range(6)]
            if all(feature == 'Missing' for feature in bfp_arr):
                simple_tool[attack_type[raw_data[4]]] += float(raw_data[6])
            elif any(name in raw_data[3] for name in botvm) or any(name in raw_data[2] for name in botvm):
                vm_tool[attack_type[raw_data[4]]] += float(raw_data[6])
            else:
                browser_tool[attack_type[raw_data[4]]] += float(raw_data[6])
    
    for ele in simple_tool:
        if ele in tools:
            tools[ele][0] += simple_tool[ele]
    for ele in browser_tool:
        if ele in tools:
            tools[ele][1] += browser_tool[ele]
    for ele in vm_tool:
        if ele in tools:
            tools[ele][2] += vm_tool[ele]
    
    min_cal_val = 10
    print('================================================================')
    print(company[company_num])
    for ele in bfp_dic:
        if len(bfp_dic[ele]) > min_cal_val:
            sum_attack = 0
            sum_normal = 0
            adv_bfp = 0
            com_bfp = 0
            for val in bfp_dic[ele]:
                sum_attack += bfp_dic[ele][val][0]
                sum_normal += bfp_dic[ele][val][1]
                if bfp_dic[ele][val][0] > 0:
                    com_bfp += 1
                    if bfp_dic[ele][val][1] == 0:
                        adv_bfp += 1
            print(ele, 'attack', sum_attack,'normal', sum_normal)
            if com_bfp != 0:
                print("adv_bfp",adv_bfp,"com_bfp",com_bfp,"Unique rate",adv_bfp / com_bfp)
            if ele not in unique_dic:
                unique_dic[ele] = [[], []]
            unique_dic[ele][0].append(adv_bfp)
            unique_dic[ele][1].append(com_bfp)
            k_l = 0
            for val in bfp_dic[ele]:
                if bfp_dic[ele][val][1] != 0 and bfp_dic[ele][val][0] != 0:
                    per_attack = bfp_dic[ele][val][0]/float(sum_attack)
                    per_normal = bfp_dic[ele][val][1]/float(sum_normal)
                    k_l += per_attack * np.log(per_attack/per_normal)
            print('k_l', k_l)
            if ele not in kl_dic:
                kl_dic[ele] = [k_l]
            else:
                kl_dic[ele].append(k_l)
            
        
    
print('********************************************************************')
for ele in tools:
    s_val = sum(tools[ele])
    if s_val != 0:
        print(ele, "***************************")
        print('simple:', tools[ele][0]* 1.0 / s_val  )
        print('browser:', tools[ele][1]* 1.0 / s_val  )
        print('VM:', tools[ele][2]* 1.0 / s_val  )
print('********************************************************************')
for ele in kl_dic:
    print(ele, "***************************")
    mean = sum(kl_dic[ele]) / len(kl_dic[ele])
    std = np.std(kl_dic[ele])
    print(mean, std)
print('********************************************************************')
for ele in unique_dic:
    print(ele, "***************************")
    print(sum(unique_dic[ele][0]) /sum(unique_dic[ele][1]) )
    
    


        
        